In [1]:
import wget
URL = 'http://ovne.org/download/Customer-Churn-Records.csv'
response = wget.download(URL, "Customer-Churn-Records.csv")

In [2]:
import pandas as pd
df = pd.read_csv("Customer-Churn-Records.csv", header = 0, na_values = "NA", comment='\t', sep=',', skipinitialspace=True)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   RowNumber           10000 non-null  int64  
 1   CustomerId          10000 non-null  int64  
 2   Surname             10000 non-null  object 
 3   CreditScore         10000 non-null  int64  
 4   Geography           10000 non-null  object 
 5   Gender              10000 non-null  object 
 6   Age                 10000 non-null  int64  
 7   Tenure              10000 non-null  int64  
 8   Balance             10000 non-null  float64
 9   NumOfProducts       10000 non-null  int64  
 10  HasCrCard           10000 non-null  int64  
 11  IsActiveMember      10000 non-null  int64  
 12  EstimatedSalary     10000 non-null  float64
 13  Exited              10000 non-null  int64  
 14  Complain            10000 non-null  int64  
 15  Satisfaction Score  10000 non-null  int64  
 16  Card 

In [5]:
df.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Complain', 'Satisfaction Score', 'Card Type', 'Point Earned'], inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [7]:
X = df.drop(["Exited"], axis=1)
y = df.Exited

In [8]:
dict = {"France": 0, "Germany": 1, "Spain": 2, "Female": 0, "Male": 1}
X.replace(dict, inplace = True)

/var/folders/bj/4zbr51sj4fsbvtllcchl3wzc0000gn/T/ipykernel_5265/558804013.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X.replace(dict, inplace = True)


In [9]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0,96270.64
9996,516,0,1,35,10,57369.61,1,1,1,101699.77
9997,709,0,0,36,7,0.00,1,0,1,42085.58
9998,772,1,1,42,3,75075.31,2,1,0,92888.52


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=43)


In [11]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(min_samples_leaf=10)

In [12]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_train, y_train, cv=10)

In [13]:
scores

array([0.835  , 0.8475 , 0.8325 , 0.83   , 0.8425 , 0.8275 , 0.83125,
       0.81625, 0.84875, 0.84375])

In [14]:
print('Average score: ', scores.mean())

Average score:  0.8355


In [15]:
from sklearn.model_selection import cross_validate
cv_results = cross_validate(clf, X_train, y_train, cv=10, return_estimator=True)

In [16]:
type(cv_results)

dict

In [17]:
cv_results.keys()

dict_keys(['fit_time', 'score_time', 'estimator', 'test_score'])

In [18]:
cv_results['test_score']

array([0.83875, 0.8475 , 0.8325 , 0.8275 , 0.84   , 0.8325 , 0.83125,
       0.81875, 0.84875, 0.84375])

In [19]:
print('Mean score: ', cv_results['test_score'].mean())

Mean score:  0.836125


In [20]:
cv_results['estimator'][0]

DecisionTreeClassifier(min_samples_leaf=10)

In [21]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
scoring = ['roc_auc', 'accuracy', 'f1']
scores = cross_validate(clf, X_train, y_train, cv=10, scoring=scoring)


In [22]:
scores.keys()

dict_keys(['fit_time', 'score_time', 'test_roc_auc', 'test_accuracy', 'test_f1'])

In [23]:
scores['test_roc_auc'].mean()

np.float64(0.7884583768373196)

In [24]:
scores['test_accuracy'].mean()

np.float64(0.836375)

In [25]:
scores['test_f1'].mean()

np.float64(0.5520139154327268)

In [26]:
testscores = []
for i in range(len(cv_results['estimator'])):
    testscores.append(cv_results['estimator'][i].score(X_test, y_test))

In [27]:
testscores

[0.8265, 0.836, 0.8355, 0.8475, 0.8385, 0.8235, 0.839, 0.8335, 0.8365, 0.8455]

In [28]:
y_pred = cv_results['estimator'][0].predict(X_test)

In [29]:
probabilities = cv_results['estimator'][0].predict_proba(X_test)
print(probabilities)

[[1.         0.        ]
 [0.4        0.6       ]
 [0.94444444 0.05555556]
 ...
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]]
